In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
pdf_loader = PyPDFDirectoryLoader("data")
pdfdocs = pdf_loader.load()
pdfdocs

[Document(metadata={'source': 'data/nutrients in plants.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'data/nutrients in plants.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'data/nutrients in plants.pdf', 'page': 2}, page_content=''),
 Document(metadata={'source': 'data/nutrients in plants.pdf', 'page': 3}, page_content=''),
 Document(metadata={'source': 'data/nutrients in plants.pdf', 'page': 4}, page_content=''),
 Document(metadata={'source': 'data/acid,bases,salts.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'data/acid,bases,salts.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'data/acid,bases,salts.pdf', 'page': 2}, page_content=''),
 Document(metadata={'source': 'data/acid,bases,salts.pdf', 'page': 3}, page_content=''),
 Document(metadata={'source': 'data/acid,bases,salts.pdf', 'page': 4}, page_content=''),
 Document(metadata={'source': 'data/weather, climate and adaptation of animal to climate.pdf', 

In [2]:
## Web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://learnflix.in/ncert-syllabus-class-7-science.html",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")
                     ))
                     )

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
webdocs=loader.load()

In [4]:
## Arxiv
from langchain_community.document_loaders import ArxivLoader
arxivdocs = ArxivLoader(query="2304.11149", load_max_docs=2).load()


In [5]:
docs=arxivdocs+webdocs+pdfdocs
docs

[Document(metadata={'Published': '2023-04-21', 'Title': 'Newspaper Story Problems and Other Tasks for Context Based Physics Education: A Research Based report on classroom practice', 'Authors': 'Jochen Kuhn, Andreas Müller, Patrik Vogt', 'Summary': 'Background: Context Based Science Education (CBSE) has a long-standing\ntradition and is discussed as a highly promising approach in science education.\nIt is supposed that CBSE can foster pupils engagement and learning. However,\nclassroom implementations of CBSE based on solid empirical evidence are\nsurprisingly scarce.\n  Purpose: The present research-based report of practice seeks to bridge this\ntheory practice gap for some specific forms of CBSE. We examine the use of\nscience problems based on newspaper articles and the real-life contexts they\nprovide (newspaper story problems, NSP).\n  Design and Methods: While the research background has been reported\nelsewhere, the main objective of the present contribution is to provide a\ndet

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
final_documents=text_splitter.split_documents(docs)

In [7]:
print(final_documents[0])
print(final_documents[1])

page_content='NEWSPAPER STORY PROBLEMS AND OTHER TASKS FOR CONTEXT 
BASED PHYSICS EDUCATION:  
A RESEARCH BASED REPORT ON CLASSROOM PRACTICE 
 
Kuhn, Jochen1; Müller, Andreas2; Vogt, Patrik 
1 Ludwig-Maximilians-Universität München / LMU Munich 
2 University of Geneva (andreas.mueller@unige.ch) 
 
ABSTRACT 
 
Background: Context Based Science Education (CBSE), that is using authentic, real-life contexts, has a long-standing' metadata={'Published': '2023-04-21', 'Title': 'Newspaper Story Problems and Other Tasks for Context Based Physics Education: A Research Based report on classroom practice', 'Authors': 'Jochen Kuhn, Andreas Müller, Patrik Vogt', 'Summary': 'Background: Context Based Science Education (CBSE) has a long-standing\ntradition and is discussed as a highly promising approach in science education.\nIt is supposed that CBSE can foster pupils engagement and learning. However,\nclassroom implementations of CBSE based on solid empirical evidence are\nsurprisingly scarce.\n  Pur

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")
embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x11e9ae1d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11beec8d0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [11]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(final_documents,embeddings)
db